In [60]:
# command line 
# conda install ipywidgets 
# jupyter nbextension enable --py widgetsnbextension

import os
import numpy as np
import src.ImagePreprocessFilters as IPrep
import src.ImageParser as IP
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
from glob import glob
import random
from IPython.display import display
from ipywidgets import interact
import ipywidgets as widgets
import skimage
import pathlib

In [2]:
def set_params(res=300):
    plt.rcParams['figure.dpi'] = res
    plt.rcParams['savefig.dpi'] = res
set_matplotlib_formats('svg')
set_matplotlib_formats('retina')
cmap = plt.cm.gray   # for black and white. If None get the ones purple 
set_params(res=300)

/tmp/ipykernel_1233739/3197820359.py:4: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')
/tmp/ipykernel_1233739/3197820359.py:5: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [3]:
def preprocess_images(files):
    # imgs can be one or muliple channels 
    # PARSE images to get numpy arrays into shape (, ,n_channels)
    images = map(IP.parse_image, files)
    images = list(images)
    # PERCENTILE SATURATION OUTLIERS
    up_limit = 99  # 99
    down_limit = 1  # 1
    imgsOut = map(lambda p: IPrep.remove_outliers(p, up_limit, down_limit), images)
    # NORMALIZE PER CHANNEL with function from OpenCV
    imgs_norm = map(IPrep.normalize_channel_cv2_minmax, imgsOut)
    imgs_norm = list(imgs_norm)  # to be used more again
    return images, imgs_norm

In [23]:

# PERCENTILE FILTERS PER CHANNEL 
def calculate_per_channel_percentile(images_channel,PERCENTILE):
    print(PERCENTILE)
    imgs_filtered = map(lambda p: IPrep.percentile_filter(p, window_size=3, percentile=PERCENTILE, transf_bool=True), images_channel)
    imgs_filtered = list(imgs_filtered)
    return imgs_filtered

def calculate_per_channel_consecutivepercentile(images_channel,PERCENTILE):
    imgs_filtered = map(lambda p: IPrep.percentile_filter(p, window_size=3, percentile=PERCENTILE, transf_bool=True), images_channel)
    imgs_filtered = map(lambda p: IPrep.percentile_filter(p, window_size=3, percentile=PERCENTILE, transf_bool=True), imgs_filtered)
    imgs_filtered = list(imgs_filtered)
    return imgs_filtered

def plot_one_channel(images):
    for idx in range(len(images)):
        if images[0].shape[0]==1:
            images = [images[i][0] for i in range(len(images))] 
 
        plt.imshow(images[idx][...], cmap = cmap)
        plt.show()
        
def plot_one_channel_side_by_side(images, columns=1, figsize=(20,20)):
    rows = len(images)//columns 
    fig, axs = plt.subplots(rows, columns, figsize=figsize)
    if images[0].shape[0]==1:
        images = [images[i][0] for i in range(len(images))] 

    for i, filename in enumerate(images):
        axs.flat[i].imshow(filename, cmap=cmap)
        axs.flat[i].axis('off')
    plt.tight_layout()
    plt.show()


    
def plot_compare_images(images1, images2):
    figsize=(20,20*len(images1))
#     figsize=(20,20, 20*len(images1))
    if images1[0].shape[0]==1:
        images1 = [images1[i][0] for i in range(len(images1))]
    if images2[0].shape[0]==1:
        images2 = [images2[i][0] for i in range(len(images2))]
    
    fig, axs = plt.subplots(len(images1), 2, figsize = figsize)
    for i, (image1, image2) in enumerate(zip(images1, images2)):
        axs[i, 0].imshow(image1, cmap=cmap)# , vmin=0, vmax=1
        axs[i, 0].axis('off')
        axs[i, 1].imshow(image2, cmap=cmap)
        axs[i, 1].axis('off')
    plt.tight_layout()
    plt.show()
    

In [5]:
# https://datacarpentry.org/image-processing/05-creating-histograms/
# Do histogram of a cohort per channel 
def histogram_one_channel_one_img(img):
    histogram, bin_edges = np.histogram(img, bins=256, range=(0, 1))
    # configure and draw the histogram figure
    plt.figure()
    plt.title("Grayscale Histogram")
    plt.xlabel("grayscale value")
    plt.ylabel("pixel count")
    plt.xlim([0.0, 1.0])  # <- named arguments do not work here

    plt.plot(bin_edges[0:-1], histogram)  # <- or here
    
    
def histogram_all_channel_one_img(img):
    from random import randint
    CH = img.shape[2]
    # tuple to select colors of each channel line
    colors = []
    for i in range(CH):
        colors.append('#%06X' % randint(0, 0xFFFFFF))
    # create the histogram plot, with three lines, one for
    # each color
    plt.figure()
    for channel_id, color in enumerate(colors):
        histogram, bin_edges = np.histogram(
            img[:, :, channel_id], bins=256, range=(0, 1)
        )
        plt.plot(bin_edges[0:-1], histogram, color=color)
#     plt.hist(img.ravel(), bins=256, range=(0.0, 1.0), fc='k', ec='k') #calculating histogram


def histogram_one_channel_all_img(images,linewidth=0.5):

    fig, ax = plt.subplots(figsize=(15, 7))
    for image in images:
        hist, bin_edges = np.histogram(image, bins=100, range=(0, 1))
        ax.plot(bin_edges[0:-1], hist, linewidth=linewidth)
    plt.ylim([0.0,75000])
    plt.show()


In [13]:

def calculate_psnr_snr(image1_true, image2_test):
    # Assumes image1 and image2 are numpy arrays with the same shape and dtype
    # do this by channel
    mse = np.mean((image1_true - image2_test) ** 2, axis=(0, 1))
    snr = np.mean(image1_true ** 2, axis=(0, 1)) / mse
    psnr = 10 * np.log10(np.amax(image1_true) ** 2 / mse)
    print(psnr)
    return psnr


def plot_psnr(imgs_channel,norm_imgs_channel, imgs_filtered):
        psnr = list(map(lambda p, f: skimage.metrics.peak_signal_noise_ratio(p, f, data_range = 1),
           norm_imgs_channel, imgs_filtered))
        print(psnr)
        print('original', np.max(imgs_channel[0]), np.min(imgs_channel[0]))
        print('norm', np.max(norm_imgs_channel[0]), np.min(norm_imgs_channel[0]))
        print('filtered', np.max(imgs_filtered[0]), np.min(imgs_filtered[0]))
        plt.plot(psnr) 

FOR IMAGES THAT ARE A STACK OF TIFFs
Each image is a stack of channels 

Folder structure:

-- Folder

    -- Stack_Pat1
    
    -- Stack_Pat2
    
    -- Stack_PatN
    
The channel is an index (from 0 to N channels) 


In [7]:
path = '/home/martinha/PycharmProjects/phd/Preprocess_IMC/data/metabric22'

# get_list of images to do
list_img_path = os.listdir(path)[1:10] # here just doing 10 images
# read images
files = [str(path + '/' + sub) for sub in list_img_path if 'tif' in sub]

images_original, imgs_norm = preprocess_images(files)
print(len(imgs_norm))
print(imgs_norm[0].shape)
# do the percentile all images at time 
# imgs_norm2 = map(lambda p: IPrep.out_ratio(p, th=0.4), imgs_norm)
# PERCENTILE FILTERS
# imgs_filtered = map(lambda p: IPrep.percentile_filter(p, window_size=3, percentile=50, transf_bool=True), imgs_norm)


9
(502, 541, 39)


In [24]:
# images_original
output = widgets.Output()
plot_output_histogram = widgets.Output()
plot_output_images = widgets.Output()
plot_output_comparison = widgets.Output()
plot_output_psnr = widgets.Output()

# output.clear_output()
plot_output_histogram.clear_output()
plot_output_images.clear_output()
plot_output_comparison.clear_output()
plot_output_psnr.clear_output()

# define dropdown plots images
dropdown_cols = widgets.Dropdown(options = [1,2,3,4,5], description='Columns to display:')
dropdown_res = widgets.Dropdown(options = [50,100,200,300,400,500], description='Image resolution:')
dropdown_sample_images = widgets.Dropdown(options = ['top', 'bottom', 'random', 'all'], description='Display a subset of images:')
dropdown_sample_images_number = widgets.Dropdown(options = range(2,20,2), description='Display a subset of images:')

def plot_percentile(images, COL_PLOT=2, RES =300, sample_images= 'random',sample_images_number=10):
    set_params(res=RES)
#     norm_imgs_channel
    if sample_images == 'random':
        images = random.sample(images,sample_images_number)
    elif sample_images == 'top':
        images = images[:sample_images_number]
    elif sample_images == 'bottom':
        images = images[-sample_images_number:]
    
    if COL_PLOT>1:
        plot_one_channel_side_by_side(images, columns=COL_PLOT, figsize=(50,50))
    else:
        plot_one_channel(images) 

        
        
# define dropdown comparison images
dropdow_yes_no_compare = widgets.RadioButtons(options = ['Yes', 'No'], description='Compare:')
# dropdown_i_to_compare = widgets.Checkbox(options = ['Original', 'Sat_normalized', 'denoised'], description='What to compare:')
dropdown_res_compare = widgets.Dropdown(options = [50,100,200,300,400,500], description='Image resolution:')
dropdown_sample_images_compare = widgets.Dropdown(options = ['top', 'bottom', 'random', 'all'], description='Display a subset of images:')
dropdown_sample_images_number_compare = widgets.Dropdown(options = range(2,20,2), description='Display a subset of images:')

def plot_comparison(images_or, imgs_filtered, res_compare, sample_images_compare, sample_images_number_compare):
    set_params(res=res_compare)
    
    if sample_images_compare == 'random':
        randomlist = random.sample(range(0, len(imgs_filtered)),sample_images_number_compare)
        images_or = [images_or[i] for i in randomlist]
        imgs_filtered = [imgs_filtered[i] for i in randomlist]
#         norm_imgs_channel = norm_imgs_channel[randomlist]
        
    elif sample_images_compare == 'top':
        images_or = images_or[:sample_images_number_compare]
        imgs_filtered = imgs_filtered[:sample_images_number_compare]
#         norm_imgs_channel = norm_imgs_channel[:sample_images_number]

    elif sample_images_compare == 'bottom':
        images_or = images_or[-sample_images_number_compare:]
        imgs_filtered = imgs_filtered[-sample_images_number_compare:]
#         norm_imgs_channel = norm_imgs_channel[:-sample_images_number]
    
    plot_compare_images(images_or, imgs_filtered)
       
                        
# define dropdown percentiles
dropdown_ch = widgets.Dropdown(options = range(images_original[0].shape[2]), description='Channel:')
dropdown_th = widgets.Dropdown(options = [None,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9], description='threshold:')
dropdown_percentile = widgets.Dropdown(options = [None,10,20,25,30,40,50,60,70,75,80,90,5,95,'p50consecutive'], description='Percentile:')


def do_and_plot_percentile(CH, PERCENTILE, TH, cols, res, sample_images,sample_images_number, compare):
    output.clear_output()
    plot_output_histogram.clear_output()
    plot_output_images.clear_output()  
    plot_output_comparison.clear_output()  
    plot_output_psnr.clear_output()  
    
    if isinstance(CH, int):
        imgs_channel = [images_original[i][np.newaxis,...,CH] for i in range(len(images_original))] 
        norm_imgs_channel = [imgs_norm[i][np.newaxis,...,CH] for i in range(len(imgs_norm))] 
    else:
        imgs_channel = [images_original[i][np.newaxis,...] for i in range(len(images_original))] 
        norm_imgs_channel = [imgs_norm[i][np.newaxis,...] for i in range(len(imgs_norm))] 
#         print(norm_imgs_channel[0].shape)

    if TH is not None:
        imgs_filtered = list(map(lambda p: IPrep.out_ratio2(p, th=TH), norm_imgs_channel))
    else: 
        imgs_filtered = norm_imgs_channel

    if PERCENTILE is None:
        imgs_filtered = imgs_filtered
    elif PERCENTILE == 'p50consecutive':
        imgs_filtered = calculate_per_channel_consecutivepercentile(imgs_filtered,PERCENTILE=50)
    else:
        imgs_filtered = calculate_per_channel_percentile(imgs_filtered,PERCENTILE)
#     print('Channel', CH)
#     print('Percentile', PERCENTILE)
#     print('Thrshold', TH)
    
    with plot_output_histogram:
        set_params(res=50)
        # histogram_one_channel_one_img(imgs_norm[1][...,CH])
        # histogram_one_channel_one_img(imgs_filtered[1])

        histogram_one_channel_all_img(norm_imgs_channel)
        histogram_one_channel_all_img(imgs_filtered)

    with plot_output_images:
        plot_percentile(imgs_filtered, dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value)
    #         plot_one_channel_side_by_side(imgs_filtered, columns=4, figsize=(50,50))

    with plot_output_comparison:
        plot_comparison(imgs_channel, imgs_filtered, dropdown_res.value, dropdown_sample_images.value,
                    dropdown_sample_images_number.value)
            
#     with plot_output_psnr:
#         plot_psnr(imgs_channel,norm_imgs_channel, imgs_filtered)
    

    return imgs_channel,norm_imgs_channel,imgs_filtered

def dropdown_ch_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(change.new,dropdown_percentile.value,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value,dropdow_yes_no_compare.value )
def dropdown_th_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch.value, dropdown_percentile.value, change.new,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value,
                                                                         dropdow_yes_no_compare.value)
def dropdown_percentile_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch.value,change.new,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value,
                                                                          dropdow_yes_no_compare.value)   

dropdown_ch.observe(dropdown_ch_eventhandler, names='value')
dropdown_th.observe(dropdown_th_eventhandler, names='value')
dropdown_percentile.observe(dropdown_percentile_eventhandler, names='value')


# eficient way thta just call the plot method
# def dropdown_cols_eventhandler(change):
#     plot_percentile(imgs_filtered, change.new, dropdown_res.value,dropdown_sample_images.value,
#                     dropdown_sample_images_number.value)
# def dropdown_res_eventhandler(change):
#     plot_percentile(imgs_filtered, dropdown_cols.value, change.new,dropdown_sample_images.value,
#                     dropdown_sample_images_number.value)
# def dropdown_sample_images_eventhandler(change):
#     plot_percentile(imgs_filtered, dropdown_cols.value, dropdown_res.value,change.new,
#                     dropdown_sample_images_number.value)
# def dropdown_sample_images_number_eventhandler(change):
#     plot_percentile(imgs_filtered, dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,change.new)
    
# dropdown_cols.observe(dropdown_cols_eventhandler, names='value')
# dropdown_res.observe(dropdown_res_eventhandler, names='value')
# dropdown_sample_images.observe(dropdown_sample_images_eventhandler, names='value')
# dropdown_sample_images_number.observe(dropdown_sample_images_number_eventhandler, names='value')


# not efficient way. everytime I change the display parameters it calculates the percentile again xb 

def dropdown_cols_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
        dropdown_ch.value,dropdown_percentile.value,dropdown_th.value,
        change.new, dropdown_res.value,dropdown_sample_images.value,dropdown_sample_images_number.value,
    dropdow_yes_no_compare.value)
    
def dropdown_res_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch.value,dropdown_percentile.value,dropdown_th.value,
            dropdown_cols.value, change.new,dropdown_sample_images.value, dropdown_sample_images_number.value,
        dropdow_yes_no_compare.value)
    
def dropdown_sample_images_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch.value,dropdown_percentile.value,dropdown_th.value,
            dropdown_cols.value, dropdown_res.value,change.new,dropdown_sample_images_number.value,
        dropdow_yes_no_compare.value)

def dropdown_sample_images_number_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch.value,dropdown_percentile.value,dropdown_th.value,
            dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,change.new,
        dropdow_yes_no_compare.value)
        
dropdown_cols.observe(dropdown_cols_eventhandler, names='value')
dropdown_res.observe(dropdown_res_eventhandler, names='value')
dropdown_sample_images.observe(dropdown_sample_images_eventhandler, names='value')
dropdown_sample_images_number.observe(dropdown_sample_images_number_eventhandler, names='value')


# Dashboard

input_widgets = widgets.HBox([dropdown_ch, dropdown_percentile,dropdown_th])
input_widgetsimages = widgets.HBox([dropdown_sample_images_number, dropdown_sample_images])
input_widgetsimages2 = widgets.HBox([dropdown_res,dropdown_cols])
# input_widgets_compare = widgets.HBox([dropdown_res_compare, dropdown_sample_images_number_compare,
#                                      dropdown_sample_images_compare])
# dropdow_yes_no_compare,
# display(input_widgets)

# create a container for the output. Tab is great for this. The 1st tab will host the dataframe and the 2nd one the graph
tab = widgets.Tab([plot_output_images, plot_output_histogram, plot_output_comparison, plot_output_psnr])
tab.set_title(0, 'Images')
tab.set_title(1, 'Histogram')
tab.set_title(2, 'Compare images')
tab.set_title(3, 'PSNR')

# display(tab)
dashboard = widgets.VBox([input_widgets, input_widgetsimages,input_widgetsimages2,tab])
display(dashboard)

# output.clear_output()




# todo 
# comaprison only if yes 
# the columns without and comparison without calculate again.
# no compare images esta sempre a mostrar a mesma imagem mesmo que mude o percentile 


# extend the cell if images big
# put little dogs or cats no jupyter 


25
25
25
50


In [9]:
# images_original
output = widgets.Output()
output.clear_output()


# get a way to put the Path 
btn = widgets.Button(description='Save')
display(btn)
def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
    print('Your images are being saved')
    
    # if doesnt exist creates
    if not os.path.exists(path_for_results):
        os.makedirs(path_for_results)

    # names of images
    names_save = [str(path_for_results + sub) for sub in list_img_path if 'tif' in sub]
    images_test = map(lambda p, f: IPrep.save_images(p, f, ch_last=True), imgs_filtered, names_save)
    it = list(images_test)
    print('Images saved')


# textbox 
path_for_results = 'ResultsPercentile/'

txtbox = widgets.Text(value='ResultsPercentile/', placeholder='Type something',
    description='PathSave:',disabled=False)


display(txtbox)
def txtbox_eventhandler(obj):
    path_for_results = obj.description
    
    # if doesnt exist creates
    if not os.path.exists(path_for_results):
        os.makedirs(path_for_results)





# apply the same percentile to all channels
Percentile = 50
imgs_filtered = map(lambda p: IPrep.percentile_filter(p, window_size=3, percentile=50, transf_bool=True), imgs_norm)

# consecutive
# imgs_filtered = map(lambda p: calculate_per_channel_consecutivepercentile(p, window_size=3, percentile=50, transf_bool=True), imgs_norm)


# apply different percentile per channels   put here a list of channels with the percentile to put then make a list and save
# p_list = [50*39]
# imgs_filtered = map(lambda p: IPrep.percentile_filter_changedpercentiles(p, window_size=3, percentile=p_list, transf_bool=True), imgs_norm)
imgs_filtered = list(imgs_filtered)

# images will be saved  in np 32

btn.on_click(btn_eventhandler)
output.clear_output()


Button(description='Save', style=ButtonStyle())

Text(value='ResultsPercentile/', description='PathSave:', placeholder='Type something')

# IMAGES PER CHANNEL - One file is a image correspondent to one channel 

Folder structure:

-- Folder
   
   -- Folder_Pat_1
       
       -- img_ch_name1
       
       -- img_ch_name2
       
       -- img_ch_nameN
    
    -- Folder_Pat_2
    
        -- img_ch_name1
       
        -- img_ch_name2
       
        -- img_ch_nameN
    ...

    
The channel is a name of the channel  

In [75]:
# images_original
output = widgets.Output()
plot_output_histogram = widgets.Output()
plot_output_images = widgets.Output()
plot_output_comparison = widgets.Output()
plot_output_psnr = widgets.Output()

# output.clear_output()
plot_output_histogram.clear_output()
plot_output_images.clear_output()
plot_output_comparison.clear_output()
plot_output_psnr.clear_output()

# define dropdown plots images
dropdown_cols = widgets.Dropdown(options = [1,2,3,4,5], description='Columns to display:')
dropdown_res = widgets.Dropdown(options = [50,100,200,300,400,500], description='Image resolution:')
dropdown_sample_images = widgets.Dropdown(options = ['top', 'bottom', 'random', 'all'], description='Display a subset of images:')
dropdown_sample_images_number = widgets.Dropdown(options = range(2,20,2), description='Display a subset of images:')

def plot_percentile(images, COL_PLOT=2, RES =300, sample_images= 'random',sample_images_number=10):
    set_params(res=RES)
#     norm_imgs_channel
    if sample_images == 'random':
        images = random.sample(images,sample_images_number)
    elif sample_images == 'top':
        images = images[:sample_images_number]
    elif sample_images == 'bottom':
        images = images[-sample_images_number:]
    
    if COL_PLOT>1:
        plot_one_channel_side_by_side(images, columns=COL_PLOT, figsize=(50,50))
    else:
        plot_one_channel(images) 

        
        
# define dropdown comparison images
dropdow_yes_no_compare = widgets.RadioButtons(options = ['Yes', 'No'], description='Compare:')
# dropdown_i_to_compare = widgets.Checkbox(options = ['Original', 'Sat_normalized', 'denoised'], description='What to compare:')
dropdown_res_compare = widgets.Dropdown(options = [50,100,200,300,400,500], description='Image resolution:')
dropdown_sample_images_compare = widgets.Dropdown(options = ['top', 'bottom', 'random', 'all'], description='Display a subset of images:')
dropdown_sample_images_number_compare = widgets.Dropdown(options = range(2,20,2), description='Display a subset of images:')

def plot_comparison(images_or, imgs_filtered, res_compare, sample_images_compare, sample_images_number_compare):
    set_params(res=res_compare)
    
    if sample_images_compare == 'random':
        randomlist = random.sample(range(0, len(imgs_filtered)),sample_images_number_compare)
        images_or = [images_or[i] for i in randomlist]
        imgs_filtered = [imgs_filtered[i] for i in randomlist]
#         norm_imgs_channel = norm_imgs_channel[randomlist]
        
    elif sample_images_compare == 'top':
        images_or = images_or[:sample_images_number_compare]
        imgs_filtered = imgs_filtered[:sample_images_number_compare]
#         norm_imgs_channel = norm_imgs_channel[:sample_images_number]

    elif sample_images_compare == 'bottom':
        images_or = images_or[-sample_images_number_compare:]
        imgs_filtered = imgs_filtered[-sample_images_number_compare:]
#         norm_imgs_channel = norm_imgs_channel[:-sample_images_number]
    
    plot_compare_images(images_or, imgs_filtered)
       
                        
# define dropdown percentiles

# Channels
dropdown_ch_name = widgets.Dropdown(options = channel_names, description='Channel:')
dropdown_th = widgets.Dropdown(options = [None,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9], description='threshold:')
dropdown_percentile = widgets.Dropdown(options = [None,10,20,25,30,40,50,60,70,75,80,90,5,95,'p50consecutive'], description='Percentile:')


def do_and_plot_percentile(CH, PERCENTILE, TH, cols, res, sample_images,sample_images_number, compare):
    output.clear_output()
    plot_output_histogram.clear_output()
    plot_output_images.clear_output()  
    plot_output_comparison.clear_output()  
    plot_output_psnr.clear_output()  
    
    files_channel = [file for file in result if str(dropdown_ch_name.value+'.ome.tiff') in str(file)]
    images_original, imgs_norm = preprocess_images(files_channel)
    
#     imgs_channel = [images_original[i][np.newaxis,...] for i in range(len(images_original))] 
#     norm_imgs_channel = [imgs_norm[i][np.newaxis,...] for i in range(len(imgs_norm))] 
    imgs_channel = [images_original[i] for i in range(len(images_original))] 
    norm_imgs_channel = [imgs_norm[i] for i in range(len(imgs_norm))] 

    if TH is not None:
        imgs_filtered = list(map(lambda p: IPrep.out_ratio2(p, th=TH), norm_imgs_channel))
    else: 
        imgs_filtered = norm_imgs_channel

    if PERCENTILE is None:
        imgs_filtered = imgs_filtered
    elif PERCENTILE == 'p50consecutive':
        imgs_filtered = calculate_per_channel_consecutivepercentile(imgs_filtered,PERCENTILE=50)
    else:
        imgs_filtered = calculate_per_channel_percentile(imgs_filtered,PERCENTILE)
    
    with plot_output_histogram:
        set_params(res=50)
        # histogram_one_channel_one_img(imgs_norm[1][...,CH])
        # histogram_one_channel_one_img(imgs_filtered[1])

        histogram_one_channel_all_img(norm_imgs_channel)
        histogram_one_channel_all_img(imgs_filtered)

    with plot_output_images:
        plot_percentile(imgs_filtered, dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value)
    #         plot_one_channel_side_by_side(imgs_filtered, columns=4, figsize=(50,50))

    with plot_output_comparison:
        plot_comparison(imgs_channel, imgs_filtered, dropdown_res.value, dropdown_sample_images.value,
                    dropdown_sample_images_number.value)
            
#     with plot_output_psnr:
#         plot_psnr(imgs_channel,norm_imgs_channel, imgs_filtered)
    print(imgs_filtered[0].shape)
    return imgs_channel,norm_imgs_channel,imgs_filtered

# def dropdown_ch_eventhandler(change):
#     imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(change.new,dropdown_percentile.value,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
#                     dropdown_sample_images_number.value,dropdow_yes_no_compare.value )
    
def dropdown_ch_name_eventhandler(change):
    
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(change.new,dropdown_percentile.value,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value,dropdow_yes_no_compare.value )

    
def dropdown_th_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value, dropdown_percentile.value, change.new,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value,
                                                                         dropdow_yes_no_compare.value)
def dropdown_percentile_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value,change.new,dropdown_th.value,dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,
                    dropdown_sample_images_number.value,
                                                                          dropdow_yes_no_compare.value)   

dropdown_ch_name.observe(dropdown_ch_name_eventhandler, names='value')
dropdown_th.observe(dropdown_th_eventhandler, names='value')
dropdown_percentile.observe(dropdown_percentile_eventhandler, names='value')

# not efficient way. everytime I change the display parameters it calculates the percentile again xb 

def dropdown_cols_eventhandler(change):
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
        dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
        change.new, dropdown_res.value,dropdown_sample_images.value,dropdown_sample_images_number.value,
    dropdow_yes_no_compare.value)
    
def dropdown_res_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
            dropdown_cols.value, change.new,dropdown_sample_images.value, dropdown_sample_images_number.value,
        dropdow_yes_no_compare.value)
    
def dropdown_sample_images_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
            dropdown_cols.value, dropdown_res.value,change.new,dropdown_sample_images_number.value,
        dropdow_yes_no_compare.value)

def dropdown_sample_images_number_eventhandler(change):
        imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(
            dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
            dropdown_cols.value, dropdown_res.value,dropdown_sample_images.value,change.new,
        dropdow_yes_no_compare.value)
        
dropdown_cols.observe(dropdown_cols_eventhandler, names='value')
dropdown_res.observe(dropdown_res_eventhandler, names='value')
dropdown_sample_images.observe(dropdown_sample_images_eventhandler, names='value')
dropdown_sample_images_number.observe(dropdown_sample_images_number_eventhandler, names='value')


# Dashboard

input_widgets = widgets.HBox([dropdown_ch_name, dropdown_percentile,dropdown_th])
input_widgetsimages = widgets.HBox([dropdown_sample_images_number, dropdown_sample_images])
input_widgetsimages2 = widgets.HBox([dropdown_res,dropdown_cols])
# input_widgets_compare = widgets.HBox([dropdown_res_compare, dropdown_sample_images_number_compare,
#                                      dropdown_sample_images_compare])
# dropdow_yes_no_compare,
# display(input_widgets)

# create a container for the output. Tab is great for this. The 1st tab will host the dataframe and the 2nd one the graph
tab = widgets.Tab([plot_output_images, plot_output_histogram, plot_output_comparison, plot_output_psnr])
tab.set_title(0, 'Images')
tab.set_title(1, 'Histogram')
tab.set_title(2, 'Compare images')
tab.set_title(3, 'PSNR')


# display(tab)
dashboard = widgets.VBox([input_widgets, input_widgetsimages,input_widgetsimages2,tab])
display(dashboard)


# get a way to put the Path 
btn = widgets.Button(description='Save')
display(btn)
def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
    print('Your images are being saved')
    
    # if doesnt exist creates
    path_for_results = txtbox.value

    if not os.path.exists(path_for_results):
        os.makedirs(path_for_results)

    # names of images
    files_channel = [file for file in result if str(dropdown_ch_name.value+'.ome.tiff') in str(file)]
    
    paths_save = [str(path_for_results + os.path.basename(os.path.dirname(sub))) for sub in files_channel]
    
    names_save = [str(path_for_results + os.path.basename(os.path.dirname(sub)) + '/' + os.path.basename(sub)) 
                      for sub in files_channel]
    for path in paths_save:
        if not os.path.exists(path):
            os.makedirs(path)
    imgs_channel,norm_imgs_channel,imgs_filtered = do_and_plot_percentile(dropdown_ch_name.value,dropdown_percentile.value,dropdown_th.value,
                                                                          dropdown_cols.value, dropdown_res.value,
                                                                          dropdown_sample_images.value,
                    dropdown_sample_images_number.value, dropdow_yes_no_compare.value)   
    images_test = map(lambda p, f: IPrep.save_images(p, f, ch_last=True), imgs_filtered, names_save)
    it = list(images_test)
    print('Images saved at ',path_for_results)
    print(imgs_filtered[0].shape)

# textbox 

txtbox = widgets.Text(value='ResultsPercentile/', placeholder='Type something',
    description='PathSave:',disabled=False)


display(txtbox)
def txtbox_eventhandler(obj):
    path_for_results = obj.description
    
# images will be saved  in np 32
btn.on_click(btn_eventhandler)
output.clear_output()



PATH = '/home/martinha/PycharmProjects/phd/Preprocess_IMC/data/Marieke2/2.ROIs_raw'
result = [y for x in os.walk(PATH) for y in glob(os.path.join(x[0], '*.ome.tiff'))]
channel_names = set([name.split("_")[-1].split(".ome.tiff")[0] for name in result])


Button(description='Save', style=ButtonStyle())

Text(value='ResultsPercentile/', description='PathSave:', placeholder='Type something')

80
(1000, 1000, 1)
Hello from the Save button!
Your images are being saved
Images saved at  ResultsPercentile_/
(502, 541, 39)


In [11]:
# #Creating the interactive dashboard
# @interact
# def do_percentile(CH = range(images_original[0].shape[2]), PERCENTILE = [1,0.5,0.1,0.2,0.25,0.3,0.4,0.6,0.7,0.75,0.8,0.9], TH = [None,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] ):
#     CH = 2
#     PERCENTILE = 0.5
#     TH = 0.4
# #     print('ORIGINAL')
#     imgs_channel = [images_original[i][np.newaxis,...,CH] for i in range(len(images_original))] 
#     # plot_one_channel(imgs_channel)
#     # plot_one_channel_side_by_side(imgs_channel, columns=2, figsize=(50,50))

# #     print('NORMALIZED')
#     norm_imgs_channel = [imgs_norm[i][np.newaxis,...,CH] for i in range(len(imgs_norm))] 
#     # plot_one_channel(norm_imgs_channel)
#     print(norm_imgs_channel[0].shape)
#     if TH is not None:
#         imgs_filtered = list(map(lambda p: IPrep.out_ratio2(p, th=TH), norm_imgs_channel))
#     else: 
#         imgs_filtered = norm_imgs_channel
        
#     if PERCENTILE is not None:
#         imgs_filtered = calculate_per_channel_percentile(imgs_filtered,PERCENTILE)
#     print('TH', 'PERCENTILE')
#     return imgs_filtered

# imgs_filtered = do_percentile()
# print(imgs_filtered[0].shape)

In [12]:
slider = widgets.IntSlider(
    min=0,
    max=images_original[0].shape[2],
    step=1,
    description='Slider:',
    value=3
)
display(slider)



@interact
def useless_button(CH = range(len(files))):
    for i in range(CH):
        print('useless button')
        

IntSlider(value=3, description='Slider:', max=39)

interactive(children=(Dropdown(description='CH', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0), Output()), _do…